In [ ]:
import random
import numpy as np

In [ ]:
# Input Param
N = np.random.randint(1,11)
M = np.random.randint(1,11)
W = np.random.randint(1,11)
#Pn = [np.random.randint(10,30) for _ in range(20)]
Pn = [round(random.uniform(10, 30), 2) for _ in range(20)]
Dnm = np.random.randint(0, 2, size=(N, M))
Swm = np.random.randint(0, 2, size=(W, M))
#Tnm = np.random.randint(1, 3, size=(N, M))
Tnm = np.round(np.random.uniform(0.01, 1, size=(N, M)), 2)
MaxMt = 8
MaxWt = 8
M_max = 1000

In [ ]:
print(N)
print(M)
print(W)

8
3
9


In [ ]:
# Objective Function
def obj_fn (P,Y):
  profit = 0
  for n in range(N):
    for m in range(M):
            profit += P[n] * Y[n][m]
    return profit

In [ ]:
def check_constraints(Y, Z, T, D, S):
    # 1. Product–machine compatibility
    for n in range(N):
        for m in range(M):
            if Y[n][m] > M_max * D[n][m]:
                return False, "Product–machine compatibility"

    # 2. Machine-time capacity
    for m in range(M):
        machine_time = sum(T[n][m] * Y[n][m] for n in range(N))
        if machine_time > 8:
            return False, "Machine-time capacity"

    # 3. Worker-time capacity
    for w in range(W):
        worker_time = sum(Z[w][m] for m in range(M))
        if worker_time > 8:
            return False, "Worker-time capacity"

    # 4. Worker–machine compatibility
    for w in range(W):
        for m in range(M):
            if Z[w][m] > 8 * S[w][m]:
                return False, "Worker–machine compatibility"

    # 5. Covering production time by workers
    for m in range(M):
        left = sum(Z[w][m] for w in range(W))
        right = sum(T[n][m] * Y[n][m] for n in range(N))
        if not np.isclose(left, right, atol=1e-2):
            return False, "Covering production time by workers"

    #6. Non-negativity & integer types
    for n in range(N):
        for m in range(M):
            if Y[n][m] < 0 or not np.issubdtype(type(Y[n][m]), np.integer):
                return False, "Non-negativity & type (Y)"
    for w in range(W):
        for m in range(M):
            if Z[w][m] < 0:
                return False, "Non-negativity (Z)"

    return True, "good"

In [ ]:
# def check_constraints(Y, Z, T, D, S):

#     # 1. Product–machine compatibility
#     for n in range(N):
#       for m in range(M):
#           if D[n][m] == 0 and Y[n][m] != 0:
#               return False, "Product–machine compatibility"

#     # 2. Machine-time capacity
#     for m in range(M):
#         machine_time = sum(T[n][m] * Y[n][m] for n in range(N))
#         if machine_time - MaxMt > 1e-6:
#             return False, "Machine-time capacity"

#     # 3. Worker-time capacity
#     for w in range(W):
#         worker_time = sum(Z[w][m] for m in range(M))
#         if worker_time - MaxWt > 1e-6:
#             return False, "Worker-time capacity"

#     # 4. Worker–machine compatibility
#     for w in range(W):
#         for m in range(M):
#             if S[w][m] == 0 and Z[w][m] > 0:
#                 return False, "Worker–machine compatibility"

#     # 5. Covering production time by workers
#     for m in range(M):
#         left = sum(Z[w][m] for w in range(W))
#         right = sum(T[n][m] * Y[n][m] for n in range(N))
#         if not np.isclose(left, right, atol=1e-4):
#             return False, "Covering production time by workers"

#     # 6. Non-negativity & integer types
#     for n in range(N):
#         for m in range(M):
#             if Y[n][m] < 0 or not np.issubdtype(type(Y[n][m]), np.integer):
#                 return False, "Non-negativity & type (Y)"
#     for w in range(W):
#         for m in range(M):
#             if Z[w][m] < 0:
#                 return False, "Non-negativity (Z)"

#     return True, "OK"

In [ ]:
# Parameters
num_agents = 20

def generate_feasible_agent():
    y = np.zeros((N, M), dtype=int)
    z = np.zeros((W, M))
    worker_total_time = [0.0] * W  # total time per worker

    for m in range(M):
        eligible_workers = [w for w in range(W) if Swm[w][m] == 1 and worker_total_time[w] < MaxWt]
        if not eligible_workers:
            continue

        # Time left on machine
        remaining_machine_time = MaxMt
        valid_products = [n for n in range(N) if Dnm[n][m] == 1]

        for n in valid_products:
            t_per_unit = Tnm[n][m]
            if t_per_unit == 0 or remaining_machine_time < t_per_unit:
                continue

            max_units = int(remaining_machine_time // t_per_unit)
            if max_units == 0:
                continue

            units = random.randint(0, max_units)
            y[n][m] = units
            required_time = units * t_per_unit

            # Distribute required_time among eligible workers
            assigned_time = 0
            allContribute = []
            for w in eligible_workers:
                if assigned_time >= required_time:
                    break

                available = MaxWt - worker_total_time[w]
                contribute = min(required_time - assigned_time, available)
                z[w][m] += contribute
                worker_total_time[w] += contribute
                assigned_time += contribute
                allContribute.append(contribute)

            if not np.isclose(assigned_time, required_time, atol=1e-3):
                # Could not assign enough time => rollback
                y[n][m] = 0
                c = 0
                for w in eligible_workers:
                    worker_total_time[w] -= allContribute[c]
                    z[w][m] -= allContribute[c]
                    c += 1
            else:
                remaining_machine_time -= required_time

    return y, z

# Step 4: Generate valid agents
agents_y = []
agents_z = []
personal_best_scores = []

while len(agents_y) < num_agents:
#for i in range(50):
    y, z = generate_feasible_agent()
    #test = check_feasibility(y, z, Tnm, Dnm, Swm)
    test, msg = check_constraints(y, z, Tnm, Dnm, Swm)
    print(msg)
    if test:
        agents_y.append(y)
        agents_z.append(z)
        objective = obj_fn(Pn, y)
        personal_best_scores.append(objective)

# Show results
for i in range(num_agents):
    print(f"\nAgent {i+1}")
    print("y:\n", agents_y[i])
    print("y:\n", agents_z[i])

good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good

Agent 1
y:
 [[ 4  5 38]
 [ 0  4 20]
 [ 0  0  0]
 [ 0  2  0]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]]
y:
 [[3.52 4.48 0.  ]
 [0.   0.   0.  ]
 [0.   0.   0.  ]
 [0.   2.59 5.41]
 [0.   0.   0.  ]
 [0.   0.   0.  ]
 [0.   0.   2.55]
 [0.   0.   0.  ]
 [0.   0.   0.  ]]

Agent 2
y:
 [[ 5  4  3]
 [ 0  8 33]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [ 5  0  0]]
y:
 [[6.5  1.5  0.  ]
 [0.   0.   0.  ]
 [0.   0.   0.  ]
 [0.   4.46 3.54]
 [0.   0.   0.  ]
 [0.   0.   0.  ]
 [0.   0.   2.43]
 [0.   0.   0.  ]
 [0.   0.   0.  ]]

Agent 3
y:
 [[ 0  5 53]
 [ 0 11  5]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0  0]
 [ 5  0  0]]
y:
 [[2.1  5.9  0.  ]
 [0.   0.   0.  ]
 [0.   0.   0.  ]
 [0.   1.85 6.15]
 [0.   0.   0.  ]
 [0.   0.   0.  ]
 [0.   0.   1.06]
 [0.   0.   0.  ]
 [0.   0.   0.  ]]

Agent 4
y:
 [[ 4  1 27]
 [ 0  8  7]
 [ 0  0  0]
 [ 0  0  0]
 [ 0  0 

In [ ]:
# PSO parameters
iterations = 100

w = 1     # Inertia
c1 = 1     # Cognitive
c2 = 1     # Social

velocities_Y = [np.zeros((N, M)) for _ in range(num_agents)]
velocities_Z = [np.zeros((W, M)) for _ in range(num_agents)]

personal_best_Y = agents_y.copy()
personal_best_Z = agents_z.copy()

# Global best
g_best_index = np.argmax(personal_best_scores)
global_best_Y = personal_best_Y[g_best_index].copy()
global_best_Z = personal_best_Z[g_best_index].copy()

In [ ]:
# PSO Main loop
for iter in range(iterations):
    for i in range(num_agents):
        r1, r2 = np.random.rand(), np.random.rand()

        # Update velocities
        velocities_Y[i] = (w * velocities_Y[i] +
                          c1 * r1 * (personal_best_Y[i] - agents_y[i]) +
                          c2 * r2 * (global_best_Y - agents_y[i]))

        velocities_Z[i] = (w * velocities_Z[i] +
                          c1 * r1 * (personal_best_Z[i] - agents_z[i]) +
                          c2 * r2 * (global_best_Z - agents_z[i]))



        temp_y = np.round(agents_y[i] + velocities_Y[i]).astype(int)
        temp_z = agents_z[i] + velocities_Z[i]

        # Check constraints
        print(check_constraints(temp_y, temp_z, Tnm, Dnm, Swm))
        cond, a = check_constraints(temp_y, temp_z, Tnm, Dnm, Swm)
        if not cond:
            continue  # Skip invalid solutions
        # print("hena")

        # Update positions
        agents_y[i] = temp_y
        agents_z[i] = temp_z

        # Clip to valid ranges
        #agents_y[i] = np.clip(agents_y[i], 0, M_max)
        #agents_z[i] = np.clip(agents_z[i], 0, MaxWt)
        # c2=c2*lamda
        # c1=c1*lamda


        # Evaluate and update personal best
        score = obj_fn(Pn, agents_y[i])
        if score > personal_best_scores[i]:
            personal_best_Y[i] = agents_y[i].copy()
            personal_best_Z[i] = agents_z[i].copy()
            personal_best_scores[i] = score

        # Update global best
        g_best_index = np.argmax(personal_best_scores)
        global_best_Y = personal_best_Y[g_best_index].copy()
        global_best_Z = personal_best_Z[g_best_index].copy()

    print(f"Iteration {iter+1}, Best Profit: {personal_best_scores[g_best_index]}")

# Final output
print("\n=== Final Best Solution ===")
print("Best Profit:", personal_best_scores[g_best_index])
print("Y (Units Produced):")
print(global_best_Y)
print("Z (Worker Time Allocation):")
print(np.round(global_best_Z, 2))

(True, 'good')
(True, 'good')
(False, 'Covering production time by workers')
(False, 'Covering production time by workers')
(True, 'good')
(True, 'good')
(True, 'good')
(False, 'Covering production time by workers')
(True, 'good')
(False, 'Covering production time by workers')
(True, 'good')
(False, 'Worker-time capacity')
(True, 'good')
(True, 'good')
(False, 'Worker-time capacity')
(False, 'Worker-time capacity')
(False, 'Covering production time by workers')
(False, 'Worker-time capacity')
(True, 'good')
(False, 'Covering production time by workers')
Iteration 1, Best Profit: 1859.6200000000001
(True, 'good')
(True, 'good')
(False, 'Machine-time capacity')
(False, 'Covering production time by workers')
(False, 'Worker-time capacity')
(True, 'good')
(True, 'good')
(False, 'Covering production time by workers')
(True, 'good')
(False, 'Covering production time by workers')
(True, 'good')
(True, 'good')
(True, 'good')
(True, 'good')
(True, 'good')
(False, 'Worker-time capacity')
(False,